In [105]:
# parameters
maxlen = 10
shapes = [(1, 2, 3), (1,), (1,), (1, 2, 3), (1,)]

# internal

# dynamic
flatshapes = [int(np.prod(s)) for s in shapes]


def transition():
    X = np.random.randn(*shapes[0])
    A = np.random.randint(4, size=shapes[1])
    R = np.random.randn(*shapes[2])
    X_next = np.random.randn(*shapes[3])
    I_next = np.random.randn(*shapes[4])
    return X, A, R, X_next, I_next

X, A, R, X_next, I_next = transition()


        

ec = ExperienceReplayCache(10)

for _ in range(15):
    ec.append(*transition())
    
ec.sample()[4].shape

(1,)

In [106]:
ec.dataframe


X                                                      A         R  \
          0         1         2         3         4         5    0         0   
0 -0.813442 -0.215406  0.221687 -1.256796 -0.713318  1.082547  0.0 -0.199198   
1  0.074225 -0.898789 -2.658496  0.924917  0.621941 -1.964053  3.0 -2.827992   
2  0.715622 -1.227566 -0.810555 -0.319711 -0.005923 -0.579567  3.0  0.192157   
3  0.318597 -0.997735 -0.361035 -0.556761 -0.029555 -0.009637  3.0  0.303857   
4 -1.195276  0.526298  0.844295 -0.796267 -0.935284  0.657051  2.0  0.834397   
5 -0.202399 -0.101890 -1.288584 -0.291521 -0.327138 -0.790830  2.0  1.296568   
6  0.392259 -0.536311 -2.351972  2.144741 -0.144224 -0.216513  2.0 -2.805952   
7  2.178274  0.429239  0.495341  0.654944 -1.928511  0.215664  3.0  1.129600   
8  0.075767 -0.767861 -0.951861 -0.126230 -0.352005  0.265372  3.0  0.372601   
9 -1.106901 -0.274816  0.379836 -1.860804  0.528951  0.923909  3.0 -0.325665   

     X_next                                                      I_next  
          0         1         2         3         4         5         0  
0 -1.053526  2.050481 -0.670752 -1.281286  0.426127  1.213456 -0.672282  
1 -0.344952  1.275855 -1.893048 -2.464114 -0.597428  0.388893 -0.748152  
2  1.021456  0.039563 -0.282796  0.943431  1.853933 -2.611702  0.760975  
3 -0.421659  1.508838  1.756177  0.850243  1.384362  1.517050  0.525714  
4 -1.105506  1.238850 -0.294635  1.723192  1.118720  0.028243 -0.019115  
5 -1.129515 -0.676650 -0.382819  1.097037  0.142721  0.377400  0.484316  
6 -0.941389  1.360502 -0.580881  0.864618 -2.120115 -0.705449 -0.048230  
7 -0.801597  0.769720  0.829552  0.710612 -0.066956  0.685388 -1.195267  
8 -0.132532  1.203291 -0.235542  1.518281 -0.906658 -1.175103  0.246547  
9 -0.255404  0.229369  0.521172 -0.879141 -0.050293  0.549605  0.225962